In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-01"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
15088,2024-03-01,Finlândia I Divisão A,14:00,Raiders Basket,HBA-Marsky Helsinki,1.22,3.98,167.5,1.80,1.86,0.0,0.00,0.0,h04aRCXF,0.819672,0.251256,0.555556,0.537634,0.070928,214.390,146.847212,0.684954,1.2,1.643168,1.369306,465.06,2.684,2.330790,0.868401,18.0,399.576,287.399884,0.719262,0.6,1.341641,2.236068,232.80,5.770,4.336859,0.751622,-62.0,69,80,6.74,2.91,246.966,378.966,0.750621,0.023184,NaN,-2.50,-0.500,-0.440000,0.000000,0.0,0.000000,-2.83,-0.566,-5.265018,0.000000,0.0,0.000000
15089,2024-03-01,Finlândia Korisliiga Feminina,13:30,HBA-Marsky Helsinki F,Espoo Team F,1.66,2.13,144.5,1.80,1.90,0.0,0.00,0.0,K09bFsym,0.602410,0.469484,0.555556,0.526316,0.071893,384.660,273.553484,0.711157,0.6,1.341641,2.236068,756.49,5.224,2.887591,0.552755,-18.0,498.654,480.121858,0.962836,1.2,1.643168,1.369306,155.94,6.982,7.250988,1.038526,-57.0,101,69,7.49,2.26,274.504,533.418,0.175377,0.038222,NaN,-2.84,-0.568,-1.161972,0.000000,0.0,0.000000,6.17,1.234,0.915721,0.000000,0.0,0.000000
15090,2024-03-01,Finlândia Korisliiga Feminina,13:30,Kouvottaret F,Vimpelin Veto F,1.39,2.84,158.5,1.83,1.87,0.0,0.00,0.0,WzD2E1jg,0.719424,0.352113,0.546448,0.534759,0.071537,418.398,407.750540,0.974552,1.2,1.643168,1.369306,767.25,4.566,4.064410,0.890147,23.0,143.346,79.846014,0.557016,1.8,1.643168,0.912871,71.69,1.706,0.788372,0.462117,-4.0,93,67,8.25,1.07,297.276,198.116,0.484778,0.015289,NaN,6.07,1.214,0.321252,0.000000,0.0,0.000000,0.87,0.174,10.574713,0.000000,0.0,0.000000
15091,2024-03-01,França Pro B,16:00,Aix-Maurienne,Evreux,1.50,2.47,160.5,1.85,1.85,0.0,0.00,0.0,Ucqr6rEo,0.666667,0.404858,0.540541,0.540541,0.071525,238.460,73.460084,0.308060,0.6,1.341641,2.236068,320.45,2.892,0.755526,0.261247,-17.0,172.962,82.908114,0.479343,0.6,1.341641,2.236068,115.73,2.368,1.223936,0.516865,-36.0,85,71,3.77,1.63,180.686,217.132,0.345538,0.000000,NaN,1.75,0.350,1.428571,0.000000,0.0,0.000000,-1.66,-0.332,-4.427711,0.000000,0.0,0.000000
15092,2024-03-01,França Pro B,16:00,La Rochelle,Denain-Voltaire,1.19,4.32,150.5,1.85,1.85,0.0,0.00,0.0,IJFPm1bH,0.840336,0.231481,0.540541,0.540541,0.071818,124.076,30.048215,0.242176,2.4,1.341641,0.559017,155.40,1.708,0.445500,0.260831,38.0,186.784,91.131855,0.487900,1.2,1.643168,1.369306,102.90,2.452,0.960947,0.391904,-21.0,70,70,2.22,1.47,98.240,255.504,0.803355,0.000000,NaN,1.21,0.242,0.785124,0.000000,0.0,0.000000,-1.39,-0.278,-11.942446,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15215,2024-03-01,Coreia Do Sul Kbl,04:00,LG Sakers,Anyang JungKwanJang,1.26,3.81,164.5,1.85,1.89,0.0,0.00,0.0,S6G4qDVq,0.793651,0.262467,0.540541,0.529101,0.056118,169.508,27.700364,0.163416,1.2,1.643168,1.369306,164.16,2.096,0.319734,0.152545,-12.0,220.398,112.599503,0.510892,0.0,0.000000,NaN,324.81,2.910,1.172412,0.402891,-68.0,76,81,2.16,4.01,143.672,249.120,0.711291,0.015125,NaN,-3.80,-0.760,-0.342105,0.698453,0.4,-0.298453,-5.00,-1.000,-2.810000,0.334789,0.0,-0.334789
15216,2024-03-01,Coreia Do Sul Kbl,02:00,Seoul Knights,KoGas,1.39,2.96,155.5,1.80,1.94,0.0,0.00,0.0,lM4Dsi0d,0.719424,0.337838,0.555556,0.515464,

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
15112,22:15,Eua Ncaa,Texas State,Troy,2.28,Back Home
15191,21:00,Eua Ncaa,North Florida,Stetson,1.87,Back Home
15192,21:00,Eua Ncaa,Niagara,Rider,1.77,Back Home
15194,21:00,Eua Ncaa,Jacksonville,Florida Gulf Coast,1.80,Back Home
15197,21:00,Eua Ncaa,Georgia Southern,Old Dominion,1.55,Back Home
